In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import collections
import pickle
import gzip

from tf.app import use

In [3]:
A = use('oldbabylonian', hoist=globals(), lgc=True)

Using TF app oldbabylonian in /Users/dirk/github/annotation/app-oldbabylonian/code
Using Nino-cunei/oldbabylonian/tf - 1.0.4 in /Users/dirk/github


# Parallels

We make edges between similar lines.

When are lines similar?

If a certain distance metric is above a certain threshold.

We choose this metric:

* we reduce a line to the set of readings and graphemes in it, excluding unknown signs and ellipses.
* the similarity between two lines is the length of the intersection divided by the length of the union of their sets times 100.

# Preparation

We pre-compute all sets for all lines.

In [4]:
READABLE_TYPES = {'reading', 'grapheme', 'numeral', 'complex'}

def makeSet(l):
  if F.lnc.v(l): # comment line
    return None
  lineSet = set()
  for s in L.d(l, otype='sign'):
    if F.type.v(s) in READABLE_TYPES:
      r = F.readingr.v(s)
      if r:
        lineSet.add(r)
      g = F.graphemer.v(s)
      if g:
        lineSet.add(g)
  return lineSet

In [5]:
lines = {}

for l in F.otype.s('line'):
  lineSet = makeSet(l)
  if lineSet:
    lines[l] = lineSet
    
nLines = len(lines)
print(f'{nLines} lines')

25923 lines


# Measure

In [6]:
def sim(lSet, mSet):
  return int(round(100 * len(lSet & mSet) / len(lSet | mSet)))

# Compute all similarities

We are going to perform several hundreds of millions of comparisons, each of which is more than an elemetary operation.

Let's measure time.

In [7]:
THRESHOLD = 90

def computeSim(limit=None):
  similarity = {}

  lineNodes = sorted(lines.keys())
  nLines = len(lineNodes)

  nComparisons = nLines * (nLines - 1) // 2

  print(f'{nComparisons} comparisons to make')
  chunkSize = nComparisons // 100

  co = 0
  b = 0
  si = 0
  p = 0

  indent(reset=True)

  stop = False
  for i in range(nLines):
    nodeI = lineNodes[i]
    lineI = lines[nodeI]
    for j in range(i + 1, nLines):
      nodeJ = lineNodes[j]
      lineJ = lines[nodeJ]
      s = sim(lineI, lineJ)
      co += 1
      b += 1
      if b == chunkSize:
        p += 1
        info(f'{p:>3}% - {co:>12} comparisons and {si:>10} similarities')
        b = 0
        if limit is not None and p >= limit:
          stop = True
          break

      if s < THRESHOLD:
        continue
      similarity[(nodeI, nodeJ)] = sim(lineI, lineJ)
      si += 1
    if stop:
      break

  info(f'{p:>3}% - {co:>12} comparisons and {si:>10} similarities')
  return similarity

We are going to run it to 1% first and do some checks then.

In [8]:
similarity = computeSim(limit=1)

335988003 comparisons to make
  4.42s   1% -      3359880 comparisons and       5695 similarities
  4.42s   1% -      3359880 comparisons and       5695 similarities


We check the sanity of the results.

In [9]:
print(min(similarity.values()))
print(max(similarity.values()))

90
100


In [10]:
eq = [x for x in similarity.items() if x[1] >= 100]
neq = [x for x in similarity.items() if x[1] <= 90]

In [11]:
print(len(eq))
print(len(neq))

5689
5


In [12]:
print(eq[0])
print(neq[0])

((230788, 235394), 100)
((230797, 230811), 90)


In [13]:
A.plain(eq[0][0][0])
A.plain(eq[0][0][1])

In [14]:
A.plain(neq[0][0][0])
A.plain(neq[0][0][1])

Looks good.

Now the whole computation.

But if we have done this before, and nothing has changed, we load previous results from disk.

If we do not find previous results, we compute them and save the results to disk.

In [15]:
PARA_DIR = f'{A.tempDir}/parallels'

def writeResults(data, location, name):
  if not os.path.exists(location):
    os.makedirs(location, exist_ok=True)
  path = f'{location}/{name}'
  with gzip.open(path, 'wb') as f:
    pickle.dump(data, f)
  print(f'Data written to {path}')
  
def readResults(location, name):
  path = f'{location}/{name}'
  if not os.path.exists(path):
    print(f'File not found: {path}')
    return None
  with gzip.open(path, 'rb') as f:
    data = pickle.load(f)
  print(f'Data read from {path}')
  return data

In [16]:
similarity = readResults(PARA_DIR, f'sim-{A.version}.zip')
if not similarity:
  similarity = computeSim()
  writeResults(similarity, PARA_DIR, f'sim-{A.version}.zip')

File not found: /Users/dirk/github/Nino-cunei/oldbabylonian/_temp/parallels/sim-1.0.4.zip
335988003 comparisons to make
  4.52s   1% -      3359880 comparisons and       5695 similarities
  8.68s   2% -      6719760 comparisons and      10604 similarities
    13s   3% -     10079640 comparisons and      16028 similarities
    17s   4% -     13439520 comparisons and      24005 similarities
    21s   5% -     16799400 comparisons and      30694 similarities
    26s   6% -     20159280 comparisons and      37992 similarities
    30s   7% -     23519160 comparisons and      44479 similarities
    34s   8% -     26879040 comparisons and      49298 similarities
    38s   9% -     30238920 comparisons and      56259 similarities
    43s  10% -     33598800 comparisons and      59520 similarities
    47s  11% -     36958680 comparisons and      64009 similarities
    52s  12% -     40318560 comparisons and      69139 similarities
    56s  13% -     43678440 comparisons and      71511 similarit

In [17]:
len(similarity)

576523

So, over half a million pairs of similar lines.

Let's find out which lines have the most correspondences.

In [18]:
parallels = {}

for (l, m) in similarity:
  parallels.setdefault(l, set()).add(m)
  parallels.setdefault(m, set()).add(l)
  
print(f'{len(parallels)} out of {nLines} lines have at least one similar line')

8344 out of 25923 lines have at least one similar line


In [19]:
rankedParallels = sorted(
  parallels.items(),
  key=lambda x: (-len(x[1]), x[0]),
)

In [20]:
for (l, paras) in rankedParallels[0:10]:
  print(f'{len(paras):>4} siblings of {l} = {T.text(l)}')

1005 siblings of 230789 = qi2-bi2-[ma]
1005 siblings of 230825 = qi2-bi2-ma
1005 siblings of 230843 = [qi2]-bi2#-ma
1005 siblings of 230907 = qi2-bi2-ma
1005 siblings of 230915 = qi2-bi2-ma
1005 siblings of 231003 = qi2-bi2-ma
1005 siblings of 231021 = qi2-bi2-ma
1005 siblings of 231054 = [qi2]-bi2-ma
1005 siblings of 231067 = qi2-bi2-ma
1005 siblings of 231087 = [qi2]-bi2#-ma


In [21]:
for (l, paras) in rankedParallels[1000:1010]:
  print(f'{len(paras):>4} siblings of {T.text(l)}')

1005 siblings of qi2-bi2-ma
1005 siblings of qi2-bi2-ma
1005 siblings of qi2-bi2-ma
1005 siblings of qi2-bi2-ma
1005 siblings of qi2-bi2-ma
1005 siblings of qi2-[bi2]-ma
 128 siblings of {d}utu u3 {d}marduk li-ba-al-li-t,u2-ka
 128 siblings of {d}utu u3 {d}marduk li-ba-al-li-t,u2-ka
 128 siblings of {d}utu u3 {d}marduk li-ba-al#-li#-t,u2#-[ka]
 128 siblings of [{d}utu u3 {d}]marduk# li#-[ba]-al#-[li]-t,u2#-ka#


In [22]:
for (l, paras) in rankedParallels[1130:1140]:
  print(f'{len(paras):>4} siblings of {T.text(l)}')

 128 siblings of _{d}utu_ u3 _{d}marduk_ li-ba-al-li-t,u2-ka
 128 siblings of _{d}utu_ u3 _{d}marduk_ li-ba-al#-li#-t,u2#-ka#
 128 siblings of _{d}utu_ u3 _{d}marduk_ li-ba-al-li-t,u2-ka#
 127 siblings of {d}utu u3 {d}marduk li-ba-al-li-t,u2-ka!(KI)
 127 siblings of {d}utu u3 {d}marduk tu-ba-al-li-t,u2-ka
 125 siblings of um-ma ha-am-mu-ra-pi2-ma
 125 siblings of um-ma ha-am-mu-ra-pi2-ma
 125 siblings of um-ma ha-am-mu-ra-pi2-ma
 125 siblings of um-ma ha-am-mu#-[ra]-pi2-ma
 125 siblings of um-ma ha-am-mu#-[ra-pi2-ma]


And how many lines have just one correspondence?

We look at the tail of rankedParallels.

In [23]:
pairs = [(x, list(paras)[0]) for (x, paras) in rankedParallels if len(paras) == 1]
print(f'There are {len(pairs)} exclusively parallel pairs of lines')

There are 1994 exclusively parallel pairs of lines


Why not make an overview of exactly how wide-spread parallel lines are?

We count how many lines have how many parallels.

In [24]:
parallelCount = collections.Counter()

buckets = (2, 10, 20, 50, 100)

bucketRep = {}
prevBucket = None
for bucket in buckets:
  if prevBucket is None:
    bucketRep[bucket] = f'       n <= {bucket:>3}'
  elif bucket == buckets[-1]:
    bucketRep[bucket] = f'       n >  {bucket:>3}'
  else:
    bucketRep[bucket] = f'{prevBucket:>3} <  n <= {bucket:>3}'
  prevBucket = bucket

for (l, paras) in rankedParallels:
  clusterSize = len(paras) + 1
  if clusterSize > buckets[-1]:
    theBucket = buckets[-1]
  else:
    for bucket in buckets:
      if clusterSize <= bucket:
        theBucket = bucket
        break
  parallelCount[theBucket] += 1
  
for (bucket, amount) in sorted(
  parallelCount.items(),
  key=lambda x: (-x[0], x[1]),
):
  print(f'{amount:>4} lines have n sisters where {bucketRep[bucket]}')

1990 lines have n sisters where        n >  100
 939 lines have n sisters where  20 <  n <=  50
 775 lines have n sisters where  10 <  n <=  20
2646 lines have n sisters where   2 <  n <=  10
1994 lines have n sisters where        n <=   2


# Add parallels to the TF dataset

We can add this information to the Oldbabylonian dataset as an *edge feature*.

An edge feature links two nodes and may annotate that link with a value.

For parallels, we link each line to each of its parallel lines and we annotate that link with the similarity between
the two lines. The similarity is a percentage, and we round it to integer values.

If *n1* is similar to *n2*, then *n2* is similar to *n1*.
In order to save space, we only add such links once.

We can then use
[`E.sim.b(node)`](https://annotation.github.io/text-fabric/Api/Features/#edge-features)
to find all nodes that are parallel to node.


In [25]:
metaData = {
  '': {
    'name': 'AbB Old Babylonian Cuneiform',
    'editor': 'Cale Johnson et. al.',
    'institute': 'CDL',
    'converters': 'Cale Johnson, Dirk Roorda',    
  },
  'sim': {
    'valueType': 'int',
    'edgeValues': True,
    'description': 'similarity between lines, as a percentage of the common material wrt the combined material',
  },
}

In [26]:
simData = {}
for ((f, t), d) in similarity.items():
  simData.setdefault(f, {})[t] = d

In [27]:
ghBase = os.path.expanduser('~/github')
subdir = 'parallels'
path = f'{A.org}/{A.repo}/{subdir}/tf'
location = f'{ghBase}/{path}'
module = A.version

In [28]:
TF.save(edgeFeatures=dict(sim=simData), metaData=metaData, location=location, module=module)

   |     0.53s T sim                  to /Users/dirk/github/Nino-cunei/oldbabylonian/parallels/tf/1.0.4


True

# Turn the parallels feature into a module

The new `sim` feature is a big data feature. You do not want to load it all the time.

Here we show how to turn it into a module, so that users can easily load it in a Jupyter notebook or in the TF browser.

In [29]:
%%bash
text-fabric-zip 'Nino-cunei/oldbabylonian/parallels/tf'

True
Create release data for Nino-cunei/oldbabylonian/parallels/tf
zip files end up in /Users/dirk/Downloads/Nino-cunei-release/oldbabylonian
zipping Nino-cunei/oldbabylonian  1.0.1 with   1 features ==> parallels-tf-1.0.1.zip
zipping Nino-cunei/oldbabylonian  1.0.4 with   1 features ==> parallels-tf-1.0.4.zip


I have added this file to a new release of the Oldbabylonian Github repo.

# Use the parallels module

We load the Oldbabylonian corpus again, but now with the parallels module.

In [30]:
A = use('oldbabylonian', hoist=globals(), check=True, mod='Nino-cunei/oldbabylonian/parallels/tf')

TF app is up-to-date.
Using annotation/app-oldbabylonian commit 82fffd1ec080e9aada4018099c1ad7686317fb2c (=latest)
  in /Users/dirk/text-fabric-data/__apps__/oldbabylonian.
No new data release available online.
Using Nino-cunei/oldbabylonian/tf - 1.0.1 rv1.1 (=latest) in /Users/dirk/text-fabric-data.
No new data release available online.
Using Nino-cunei/oldbabylonian/parallels/tf - 1.0.1 rv1.1 (=latest) in /Users/dirk/text-fabric-data.


Lo and behold: you see the parallels module listed with one feature: `sim`. It is in *italics*, which indicates
it is an edge feature.

We just do a quick check here and in another notebook we study parallels a bit more, using the feature `sim`.

We count how many similar pairs their are, and how many 100% similar pairs there are.

In [31]:
query = '''
line
-sim> line
'''
results = A.search(query)

  1.55s 574579 results


In [32]:
query = '''
line
-sim=100> line
'''
results = A.search(query)

  1.58s 573397 results


Remarkably, most of the pairs are 100 percent similar. Let's show just a few:

In [33]:
A.table(results, start=1, end=10, withNodes=True)

n,p,line,line
1,P510729 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,a-na {d}suen-i-din-[nam] 235393
2,P510730 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,a-na {d}suen-i-din-nam 235420
3,P510731 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,a-na {d}suen-i-din-nam 235433
4,P510732 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,a-na {d}suen#-i-din-nam# 235463
5,P497779 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,a-na {d}suen#-[i]-din-nam 235477
6,P510733 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,[a-na] {d}[suen-i-din-nam] 235502
7,P510734 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,[a-na {d}suen-i-din-nam] 235528
8,P510736 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,a-na {d}suen-i-din-nam 235584
9,P510737 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,a-na {d}suen-i-din-nam# 235614
10,P370926 obverse:1,[a-na] _{d}suen_-i-[din-nam] 230787,a-na {d}suen-i-din-nam 235628


There is also a lower level way to work with edge features.

We can list all edges going out from a reference node.
What we see is tuple of pairs: the target node and the similarity between the reference node and that target node.

In [34]:
refNode = 235900

E.sim.f(refNode)

((235919, 100),
 (235983, 100),
 (235998, 100),
 (236046, 100),
 (236063, 100),
 (236077, 100),
 (236115, 100),
 (236123, 100),
 (236195, 100),
 (236222, 100),
 (236277, 100),
 (236320, 100),
 (243205, 100),
 (244633, 100),
 (245089, 100),
 (246839, 100),
 (246870, 100),
 (246892, 100),
 (246911, 100),
 (246924, 100),
 (246948, 100),
 (246961, 100),
 (247114, 100),
 (247136, 100),
 (247235, 100),
 (247263, 100),
 (247272, 100),
 (247317, 100),
 (247345, 100),
 (247355, 100),
 (247371, 100),
 (247386, 100),
 (247415, 100),
 (247436, 100),
 (247459, 100),
 (247542, 100),
 (247696, 100),
 (247708, 100),
 (247733, 100),
 (247749, 100),
 (247794, 100),
 (247806, 100),
 (247829, 100),
 (247867, 100),
 (247885, 100),
 (248014, 100),
 (248041, 100),
 (248118, 100),
 (248172, 100),
 (248381, 100),
 (248407, 100),
 (254854, 100),
 (256079, 100))

Likewise, we can observe the nodes that target the reference node:

In [35]:
E.sim.t(refNode)

((230787, 100),
 (235393, 100),
 (235420, 100),
 (235433, 100),
 (235463, 100),
 (235477, 100),
 (235502, 100),
 (235528, 100),
 (235584, 100),
 (235614, 100),
 (235628, 100),
 (235656, 100),
 (235683, 100),
 (235729, 100),
 (235755, 100),
 (235777, 100),
 (235810, 100),
 (235821, 100),
 (235837, 100),
 (235850, 100),
 (235864, 100),
 (235878, 100))

Both sets of nodes are similar to the reference node and it is inconvenient to use both `.f()` and `.t()` to get the similar lines.

But there is another way:

In [36]:
E.sim.b(refNode)

((230787, 100),
 (235393, 100),
 (235420, 100),
 (235433, 100),
 (235463, 100),
 (235477, 100),
 (235502, 100),
 (235528, 100),
 (235584, 100),
 (235614, 100),
 (235628, 100),
 (235656, 100),
 (235683, 100),
 (235729, 100),
 (235755, 100),
 (235777, 100),
 (235810, 100),
 (235821, 100),
 (235837, 100),
 (235850, 100),
 (235864, 100),
 (235878, 100),
 (235919, 100),
 (235983, 100),
 (235998, 100),
 (236046, 100),
 (236063, 100),
 (236077, 100),
 (236115, 100),
 (236123, 100),
 (236195, 100),
 (236222, 100),
 (236277, 100),
 (236320, 100),
 (243205, 100),
 (244633, 100),
 (245089, 100),
 (246839, 100),
 (246870, 100),
 (246892, 100),
 (246911, 100),
 (246924, 100),
 (246948, 100),
 (246961, 100),
 (247114, 100),
 (247136, 100),
 (247235, 100),
 (247263, 100),
 (247272, 100),
 (247317, 100),
 (247345, 100),
 (247355, 100),
 (247371, 100),
 (247386, 100),
 (247415, 100),
 (247436, 100),
 (247459, 100),
 (247542, 100),
 (247696, 100),
 (247708, 100),
 (247733, 100),
 (247749, 100),
 (247794

Let's make sure that `.b()` gives the combination of `.f()` and `.t()`.

In [37]:
f = {x[0] for x in E.sim.f(refNode)}
b = {x[0] for x in E.sim.b(refNode)}
t = {x[0] for x in E.sim.t(refNode)}

# are f and t disjoint ?

print(f'the intersection of f and t is {f & t}')

# is b the union of f and t ?

print(f't | f = b ? {f | t == b}')

the intersection of f and t is set()
t | f = b ? True
